In [ ]:
# import configparser
import logging
from nrbdaq.instr.ae31 import AE31
from nrbdaq.utils.sftp import SFTPClient
from nrbdaq.utils.utils import load_config, setup_logger

def main():
    config = load_config('nrbdaq.cfg')

    logger = setup_logger(name=__name__,
                          file=config['logging']['file'], 
                          level_console=config['logging']['level_console'], 
                          level_file=config['logging']['level_file'])
    logger.info('started')

    ae31 = AE31(config_file='nrbdaq.cfg')
    ae31.start()

if __name__=="__main__":
    main()


In [11]:
import os
import polars as pl

source = "nrbdaq/data/ae31"
df = pl.DataFrame()

for root, dirs, files in os.walk(source):
    for file in files:
        if df.is_empty():
            df = pl.read_csv(os.path.join(root, file), has_header=False)
        else:
            try:
                df = pl.concat([df, pl.read_csv(os.path.join(root, file), has_header=False)], how="diagonal")
            except:
                print(f"{file} could not be appended.")
                pass

df.describe()
df.write_parquet('ae31_nrb.parquet')